## Tugas 2

Preprocessing hasil crawling data dari jatim.tribunnews.com

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk mendapatkan konten dari URL
def get_soup(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')
    except requests.RequestException as e:
        print(f"Kesalahan saat mengambil {url}: {e}")
        return None

# Fungsi untuk mendapatkan detail artikel dari halaman detail
def get_article_details(detail_url):
    detail_soup = get_soup(detail_url)
    if detail_soup:
        # Ambil isi berita
        content = ' '.join([p.text for p in detail_soup.find_all('p')])

        # Ambil tanggal publikasi
        date_tag = detail_soup.find('time')
        date = date_tag.text.strip() if date_tag else 'Tidak ada tanggal'

        # Ambil judul berita
        title_tag = detail_soup.find('h1')
        title = title_tag.text.strip() if title_tag else 'Tidak ada judul'

        # Ambil kategori dari breadcrumb
        breadcrumb = detail_soup.find('ul', {'class': 'breadcrumb'})
        category = breadcrumb.find_all('li')[-1].find('span').text.strip() if breadcrumb else 'Tidak ada kategori'

        return {
            'judul': title,
            'isi_berita': content,
            'tanggal': date,
            'kategori': category,
            'url': detail_url
        }
    return None

# Fungsi untuk mendapatkan artikel dari suatu kategori
def get_articles(category_url, category_name, max_articles=50):
    articles = []
    page = 1
    
    while len(articles) < max_articles:
        url = f'{category_url}?page={page}'  # Periksa pola yang benar untuk paginasi
        print(f"Mengambil: {url}")  # Output debug
        soup = get_soup(url)
        
        if soup is None:
            break
        
        # Cari artikel di halaman
        article_list = soup.find_all('h3')  # Berdasarkan tata letak yang diamati
        
        if not article_list:
            print(f"Tidak ada artikel ditemukan di halaman {page}.")
            break
        
        for article in article_list:
            if len(articles) >= max_articles:
                break
            
            # Ambil URL detail artikel
            title_tag = article.find('a')
            detail_url = title_tag['href'] if title_tag else None
            
            if detail_url:
                if not detail_url.startswith('http'):
                    detail_url = f'https://jatim.tribunnews.com{detail_url}'
                # Buka halaman detail artikel
                article_details = get_article_details(detail_url)
                if article_details:
                    # Tambahkan ke daftar artikel
                    articles.append(article_details)
        
        page += 1
        time.sleep(2)  # Beri jeda agar tidak terlalu cepat melakukan permintaan
        
    return articles

# URL Kategori
categories = {
    'Travel': 'https://jatim.tribunnews.com/travel',
    'Sport': 'https://jatim.tribunnews.com/sport'
}

# Mengumpulkan semua data
all_articles = []
for category_name, category_url in categories.items():
    print(f"Menambang kategori {category_name}...")
    articles = get_articles(category_url, category_name, max_articles=50)
    all_articles.extend(articles)

# Simpan ke dalam DataFrame
df = pd.DataFrame(all_articles)

# Simpan ke dalam file CSV
df.to_csv('tribunnews_articles.csv', index=False)

# Tampilkan 10 data pertama dalam bentuk tabel
print(df.head(10))

print("Proses penambangan data selesai, data tersimpan dalam 'tribunnews_articles.csv' dan 10 data pertama ditampilkan.")


Menambang kategori Travel...
Mengambil: https://jatim.tribunnews.com/travel?page=1
Mengambil: https://jatim.tribunnews.com/travel?page=2
Menambang kategori Sport...
Mengambil: https://jatim.tribunnews.com/sport?page=1
Mengambil: https://jatim.tribunnews.com/sport?page=2
                                               judul  \
0  Peluang Menggiurkan Bisnis Travel, Tips dan St...   
1  TWA Kawah Ijen Dibuka, Bupati Ipuk Fiestiandan...   
2  Cicipi Tongseng Ayam Bu Mumpuni, Rekomendasi K...   
3  Jebolan Masterchef Indonesia Mario Oswin Sajik...   
4  Coba Keseruan Main Peran Laundry Staff di KidZ...   
5  Mario Oswin Hidangkan Hawaii Loco Moco Disesua...   
6  Kue Bulan Fengxian Klasik, Snow Skin Lembut hi...   
7  Legitnya Klepon Gianyar Bali Sejak 1980, Sensa...   
8  FireHouse Siap Guncang Bandung dan Surabaya, B...   
9  Nikmati Staycation Lewat Paylater Hotel dari T...   

                                          isi_berita  \
0  TRIBUNJATIM.COM - Bisnis travel menawarkan ber...   


In [8]:
df=pd.read_csv("tribunnews_articles.csv")
df.head(1000)

,judul,isi_berita,tanggal,kategori,url
0,"Peluang Menggiurkan Bisnis Travel, Tips dan St...",TRIBUNJATIM.COM - Bisnis travel menawarkan ber...,"Tayang: Selasa, 10 September 2024 15:09 WIB",Travel,https://jatim.tribunnews.com/2024/09/10/peluan...
1,"TWA Kawah Ijen Dibuka, Bupati Ipuk Fiestiandan...","Laporan Wartawan Tribun Jatim Network, Aflahul...","Tayang: Selasa, 10 September 2024 12:56 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/10/twa-ka...
2,"Cicipi Tongseng Ayam Bu Mumpuni, Rekomendasi K...","Laporan Wartawan TribunJatim.com, Sofyan Arif ...","Tayang: Senin, 9 September 2024 02:22 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/09/cicipi...
3,Jebolan Masterchef Indonesia Mario Oswin Sajik...,"Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Rabu, 4 September 2024 19:10 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/04/jebola...
4,Coba Keseruan Main Peran Laundry Staff di KidZ...,"Laporan Wartawan TribunJatim.com, Fikri Firman...","Tayang: Rabu, 4 September 2024 16:14 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/04/coba-k...
...,...,...,...,...,...
195,"Diperkuat Atlet Berpengalaman, Pickcleball Jat...","Laporan Wartawan Tribun Jatim Network, Khairul...","Tayang: Selasa, 10 September 2024 07:30 WIB",Olahraga Lain,https://jatim.tribunnews.com/2024/09/10/diperk...
196,Atlet Bowling Putri Jawa Timur Sabet Medali Em...,"Laporan Wartawan Tribun Jatim Network, Khairul...","Tayang: Senin, 9 September 2024 21:21 WIB",Olahraga Lain,https://jatim.tribunnews.com/2024/09/09/atlet-...
197,DBL Malang: Tangis Haru Kemenangan Dramatis SM...,"Laporan Wartawan Tribun Jatim Network, Rifky E...","Tayang: Senin, 9 September 2024 16:15 WIB",Basket,https://jatim.tribunnews.com/2024/09/09/dbl-ma...
198,Tim Hockey Indoor Putri Jatim Menang 10-0 dari...,"Laporan Wartawan TribunJatim.com, Khairul Amin...","Tayang: Senin, 9 September 2024 15:41 WIB",Olahraga Lain,https://jatim.tribunnews.com/2024/09/09/tim-ho...


In [10]:
import pandas as pd

df = pd.read_csv("tribunnews_articles.csv")

df.head()

,judul,isi_berita,tanggal,kategori,url
0,"Peluang Menggiurkan Bisnis Travel, Tips dan St...",TRIBUNJATIM.COM - Bisnis travel menawarkan ber...,"Tayang: Selasa, 10 September 2024 15:09 WIB",Travel,https://jatim.tribunnews.com/2024/09/10/peluan...
1,"TWA Kawah Ijen Dibuka, Bupati Ipuk Fiestiandan...","Laporan Wartawan Tribun Jatim Network, Aflahul...","Tayang: Selasa, 10 September 2024 12:56 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/10/twa-ka...
2,"Cicipi Tongseng Ayam Bu Mumpuni, Rekomendasi K...","Laporan Wartawan TribunJatim.com, Sofyan Arif ...","Tayang: Senin, 9 September 2024 02:22 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/09/cicipi...
3,Jebolan Masterchef Indonesia Mario Oswin Sajik...,"Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Rabu, 4 September 2024 19:10 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/04/jebola...
4,Coba Keseruan Main Peran Laundry Staff di KidZ...,"Laporan Wartawan TribunJatim.com, Fikri Firman...","Tayang: Rabu, 4 September 2024 16:14 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/04/coba-k...


## **Preprocessing**

Preprocessing adalah proses membersihkan dan mempersiapkan data mentah agar siap digunakan oleh model machine learning. Ini meliputi penanganan data yang hilang, normalisasi, mengubah data kategori menjadi angka, dan membersihkan teks. Tujuannya agar data lebih mudah dipahami dan diolah oleh model untuk hasil yang lebih akurat, Berikut adalah beberapa langkah umum dalam pre-processing teks:

### Cleansing

Proses cleansing data adalah tahap pembersihan teks dari elemen-elemen yang tidak relevan terhadap hasil klasifikasi sentimen. Beberapa komponen yang tidak berpengaruh terhadap sentimen, seperti URL, tag HTML, emoji, simbol, angka, dan tanda baca (~!@#$%^&*{}<>:|), dihapus dari dokumen ulasan. Elemen-elemen tersebut dihilangkan untuk mengurangi kebisingan (noise) dalam data. Setelah dihapus, elemen ini digantikan dengan spasi agar struktur kalimat tetap terjaga. Dengan demikian, data menjadi lebih fokus pada kata-kata yang relevan untuk menentukan sentimen, sehingga membantu meningkatkan akurasi model prediksi sentimen.

In [15]:
import re
import pandas as pd
import nltk
import string

def remove_url(text):
    #Fungsi untuk menghapus URL dari teks.
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_html(text):
    #Fungsi untuk menghapus tag HTML dari teks.
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

def remove_emoji(text):

    #Fungsi untuk menghapus emoji dari teks.
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emotikon wajah
        u"\U0001F300-\U0001F5FF"  # simbol & gambar
        u"\U0001F680-\U0001F6FF"  # transportasi & simbol
        u"\U0001F1E0-\U0001F1FF"  # bendera negara
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_numbers(text):
    #Fungsi untuk menghapus angka dari teks.
    return re.sub(r'\d+', '', text)

def remove_symbols(text):
    #Fungsi untuk menghapus simbol dan karakter khusus dari teks.
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# Asumsikan df adalah DataFrame yang berisi data CNN (judul, berita, tanggal, kategori)
# Contoh: df = pd.read_csv('berita-cnn.csv')

# Terapkan fungsi cleansing untuk kolom 'berita'
df['berita_clean'] = df['isi_berita'].apply(remove_url)
df['berita_clean'] = df['berita_clean'].apply(remove_html)
df['berita_clean'] = df['berita_clean'].apply(remove_emoji)
df['berita_clean'] = df['berita_clean'].apply(remove_symbols)
df['berita_clean'] = df['berita_clean'].apply(remove_numbers)



# Tampilkan beberapa baris dari hasil yang sudah dibersihkan
df.head(5)


,judul,isi_berita,tanggal,kategori,url,berita_clean
0,"Peluang Menggiurkan Bisnis Travel, Tips dan St...",TRIBUNJATIM.COM - Bisnis travel menawarkan ber...,"Tayang: Selasa, 10 September 2024 15:09 WIB",Travel,https://jatim.tribunnews.com/2024/09/10/peluan...,TRIBUNJATIMCOM Bisnis travel menawarkan berba...
1,"TWA Kawah Ijen Dibuka, Bupati Ipuk Fiestiandan...","Laporan Wartawan Tribun Jatim Network, Aflahul...","Tayang: Selasa, 10 September 2024 12:56 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/10/twa-ka...,Laporan Wartawan Tribun Jatim Network Aflahul ...
2,"Cicipi Tongseng Ayam Bu Mumpuni, Rekomendasi K...","Laporan Wartawan TribunJatim.com, Sofyan Arif ...","Tayang: Senin, 9 September 2024 02:22 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/09/cicipi...,Laporan Wartawan TribunJatimcom Sofyan Arif TR...
3,Jebolan Masterchef Indonesia Mario Oswin Sajik...,"Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Rabu, 4 September 2024 19:10 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/04/jebola...,Laporan Wartawan TribunJatimcom Nurika Anisa T...
4,Coba Keseruan Main Peran Laundry Staff di KidZ...,"Laporan Wartawan TribunJatim.com, Fikri Firman...","Tayang: Rabu, 4 September 2024 16:14 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/04/coba-k...,Laporan Wartawan TribunJatimcom Fikri Firmansy...


### CASE FOLDING

Pada tahap case folding, semua huruf kapital dalam dokumen ulasan diubah menjadi huruf kecil, atau disebut lowercase. Tujuan dari langkah ini adalah untuk menghilangkan redundansi data yang hanya disebabkan oleh perbedaan kapitalisasi. Misalnya, kata "Ekonomi" dan "ekonomi" secara teknis sama dalam analisis teks, namun tanpa case folding, komputer akan menganggapnya berbeda. Dengan mengonversi seluruh teks menjadi huruf kecil, semua variasi penulisan diseragamkan, sehingga mencegah duplikasi penghitungan atau kesalahan dalam interpretasi data.

In [16]:
def case_folding(text):
    if isinstance(text, str):
      lowercase_text = text.lower()
      return lowercase_text
    else :
      return text

df ['case_folding'] = df['berita_clean'].apply(case_folding)

df.head(5)

,judul,isi_berita,tanggal,kategori,url,berita_clean,case_folding
0,"Peluang Menggiurkan Bisnis Travel, Tips dan St...",TRIBUNJATIM.COM - Bisnis travel menawarkan ber...,"Tayang: Selasa, 10 September 2024 15:09 WIB",Travel,https://jatim.tribunnews.com/2024/09/10/peluan...,TRIBUNJATIMCOM Bisnis travel menawarkan berba...,tribunjatimcom bisnis travel menawarkan berba...
1,"TWA Kawah Ijen Dibuka, Bupati Ipuk Fiestiandan...","Laporan Wartawan Tribun Jatim Network, Aflahul...","Tayang: Selasa, 10 September 2024 12:56 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/10/twa-ka...,Laporan Wartawan Tribun Jatim Network Aflahul ...,laporan wartawan tribun jatim network aflahul ...
2,"Cicipi Tongseng Ayam Bu Mumpuni, Rekomendasi K...","Laporan Wartawan TribunJatim.com, Sofyan Arif ...","Tayang: Senin, 9 September 2024 02:22 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/09/cicipi...,Laporan Wartawan TribunJatimcom Sofyan Arif TR...,laporan wartawan tribunjatimcom sofyan arif tr...
3,Jebolan Masterchef Indonesia Mario Oswin Sajik...,"Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Rabu, 4 September 2024 19:10 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/04/jebola...,Laporan Wartawan TribunJatimcom Nurika Anisa T...,laporan wartawan tribunjatimcom nurika anisa t...
4,Coba Keseruan Main Peran Laundry Staff di KidZ...,"Laporan Wartawan TribunJatim.com, Fikri Firman...","Tayang: Rabu, 4 September 2024 16:14 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/04/coba-k...,Laporan Wartawan TribunJatimcom Fikri Firmansy...,laporan wartawan tribunjatimcom fikri firmansy...


### TOKENIZATION

Tokenization adalah tahap di mana setiap kata dalam sebuah dokumen dipecah menjadi unit-unit kata yang lebih kecil, atau disebut token. Proses ini memisahkan kata-kata berdasarkan spasi, sehingga setiap kata yang terpisah oleh spasi dianggap sebagai token tersendiri. Sebagai contoh, kalimat "Upaya agar ekonomi stabil" akan diuraikan menjadi token ["Upaya", "agar", "ekonomi", "stabil"].

In [17]:
def tokenize(text):
    tokens = text.split()
    return tokens

df['tokenize'] = df['case_folding'].apply(tokenize)

df.head(5)

,judul,isi_berita,tanggal,kategori,url,berita_clean,case_folding,tokenize
0,"Peluang Menggiurkan Bisnis Travel, Tips dan St...",TRIBUNJATIM.COM - Bisnis travel menawarkan ber...,"Tayang: Selasa, 10 September 2024 15:09 WIB",Travel,https://jatim.tribunnews.com/2024/09/10/peluan...,TRIBUNJATIMCOM Bisnis travel menawarkan berba...,tribunjatimcom bisnis travel menawarkan berba...,"[tribunjatimcom, bisnis, travel, menawarkan, b..."
1,"TWA Kawah Ijen Dibuka, Bupati Ipuk Fiestiandan...","Laporan Wartawan Tribun Jatim Network, Aflahul...","Tayang: Selasa, 10 September 2024 12:56 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/10/twa-ka...,Laporan Wartawan Tribun Jatim Network Aflahul ...,laporan wartawan tribun jatim network aflahul ...,"[laporan, wartawan, tribun, jatim, network, af..."
2,"Cicipi Tongseng Ayam Bu Mumpuni, Rekomendasi K...","Laporan Wartawan TribunJatim.com, Sofyan Arif ...","Tayang: Senin, 9 September 2024 02:22 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/09/cicipi...,Laporan Wartawan TribunJatimcom Sofyan Arif TR...,laporan wartawan tribunjatimcom sofyan arif tr...,"[laporan, wartawan, tribunjatimcom, sofyan, ar..."
3,Jebolan Masterchef Indonesia Mario Oswin Sajik...,"Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Rabu, 4 September 2024 19:10 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/04/jebola...,Laporan Wartawan TribunJatimcom Nurika Anisa T...,laporan wartawan tribunjatimcom nurika anisa t...,"[laporan, wartawan, tribunjatimcom, nurika, an..."
4,Coba Keseruan Main Peran Laundry Staff di KidZ...,"Laporan Wartawan TribunJatim.com, Fikri Firman...","Tayang: Rabu, 4 September 2024 16:14 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/04/coba-k...,Laporan Wartawan TribunJatimcom Fikri Firmansy...,laporan wartawan tribunjatimcom fikri firmansy...,"[laporan, wartawan, tribunjatimcom, fikri, fir..."


# ### STOPWORD REMOVAL

Stopword removal adalah proses menghapus kata-kata yang dianggap tidak penting atau tidak memiliki makna signifikan dalam analisis teks, seperti "dan," "di," "yang," atau "itu." Kata-kata ini sering muncul dalam kalimat tetapi tidak memberikan informasi penting untuk pemrosesan atau analisis lebih lanjut. Dengan menghapus stopwords, data teks menjadi lebih ringkas dan fokus hanya pada kata-kata yang memiliki bobot lebih besar dalam analisis, seperti saat melakukan klasifikasi atau pemodelan teks.

In [18]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rafly\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [19]:
def remove_stopwords(text):
  return [word for word in text if word not in stop_words]

df['stopword_removal'] = df['tokenize'].apply(lambda x: ' '.join(remove_stopwords(x)))


df.to_csv("preprocessing-cnnnews.csv", encoding='utf8', index=False)
df.head(5)

,judul,isi_berita,tanggal,kategori,url,berita_clean,case_folding,tokenize,stopword_removal
0,"Peluang Menggiurkan Bisnis Travel, Tips dan St...",TRIBUNJATIM.COM - Bisnis travel menawarkan ber...,"Tayang: Selasa, 10 September 2024 15:09 WIB",Travel,https://jatim.tribunnews.com/2024/09/10/peluan...,TRIBUNJATIMCOM Bisnis travel menawarkan berba...,tribunjatimcom bisnis travel menawarkan berba...,"[tribunjatimcom, bisnis, travel, menawarkan, b...",tribunjatimcom bisnis travel menawarkan peluan...
1,"TWA Kawah Ijen Dibuka, Bupati Ipuk Fiestiandan...","Laporan Wartawan Tribun Jatim Network, Aflahul...","Tayang: Selasa, 10 September 2024 12:56 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/10/twa-ka...,Laporan Wartawan Tribun Jatim Network Aflahul ...,laporan wartawan tribun jatim network aflahul ...,"[laporan, wartawan, tribun, jatim, network, af...",laporan wartawan tribun jatim network aflahul ...
2,"Cicipi Tongseng Ayam Bu Mumpuni, Rekomendasi K...","Laporan Wartawan TribunJatim.com, Sofyan Arif ...","Tayang: Senin, 9 September 2024 02:22 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/09/cicipi...,Laporan Wartawan TribunJatimcom Sofyan Arif TR...,laporan wartawan tribunjatimcom sofyan arif tr...,"[laporan, wartawan, tribunjatimcom, sofyan, ar...",laporan wartawan tribunjatimcom sofyan arif tr...
3,Jebolan Masterchef Indonesia Mario Oswin Sajik...,"Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Rabu, 4 September 2024 19:10 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/04/jebola...,Laporan Wartawan TribunJatimcom Nurika Anisa T...,laporan wartawan tribunjatimcom nurika anisa t...,"[laporan, wartawan, tribunjatimcom, nurika, an...",laporan wartawan tribunjatimcom nurika anisa t...
4,Coba Keseruan Main Peran Laundry Staff di KidZ...,"Laporan Wartawan TribunJatim.com, Fikri Firman...","Tayang: Rabu, 4 September 2024 16:14 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/04/coba-k...,Laporan Wartawan TribunJatimcom Fikri Firmansy...,laporan wartawan tribunjatimcom fikri firmansy...,"[laporan, wartawan, tribunjatimcom, fikri, fir...",laporan wartawan tribunjatimcom fikri firmansy...


## **TF-IDF (Term Frequency-Inverse Document Frequency)**

TF-IDF adalah metode statistik yang digunakan untuk mengevaluasi pentingnya suatu kata dalam sebuah dokumen relatif terhadap koleksi dokumen lainnya. TF-IDF sering digunakan dalam tugas seperti penggalian teks, penambangan informasi, dan pemodelan pembelajaran mesin berbasis teks.
Term Frequency (TF), yang menghitung seberapa sering sebuah kata muncul dalam dokumen, dan Inverse Document Frequency (IDF), yang menilai seberapa jarang kata tersebut muncul di seluruh dokumen dalam koleksi.

TF-IDF bekerja dengan memberikan bobot lebih tinggi pada kata-kata yang sering muncul dalam sebuah dokumen, tetapi jarang muncul di dokumen lain, sehingga membantu mengidentifikasi kata-kata yang paling relevan.


In [20]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv("preprocessing-cnnnews.csv")

# Menginisialisasi TfidfVectorizer
vectorizer = TfidfVectorizer()

# Menghitung TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['stopword_removal'])

In [21]:
# Mengubah hasilnya menjadi DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df.head(10)

,aaliyah,abdul,abhirama,abidin,acara,aceh,acehsumut,achmad,act,adhigana,...,yours,youtube,yuhu,yulia,yunita,yup,yusron,zhong,zona,zoo
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.036341,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
5,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
6,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
7,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
8,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.040833,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.074857,0.0
